# Voxel51 <-> V7 Integration Simple Case Walkthrough

In [ ]:
# Imports
from uuid import uuid4
import fiftyone as fo
import fiftyone.zoo as foz
import darwin_fiftyone

### Load & prepare the data

In [ ]:
# Load 10 samples of an open Voxel51 dataset
dataset = foz.load_zoo_dataset(
    "quickstart",
    max_samples=10,
    dataset_name="demonstration-dataset"
)

# Create a view of the dataset by taking 5 random samples
view = dataset.take(5)

# Display the data in the Voxel51 app
session = fo.launch_app(view)

# Define a label schema (Essentially an ontology of classes, see below for detail)
label_schema = {
    "new_detections": {
        "type": "detections",
        "classes": ["apple","orange"],
    }
}


### Upload the data to Darwin

In [ ]:
# Create a unique identifier for this annotation run
anno_key = f"key_{str(uuid4()).replace('-', '_')}"

# Send the data from Voxel51 to Darwin. 
dataset.annotate(
    anno_key,
    label_schema = label_schema,
    launch_editor=True,
    backend="darwin",
    dataset_slug="51-demo-dataset",
    base_url="https://darwin.v7labs.com/api/v2/teams",
)

**Annotation run ID**

The `anno_key` is a unique alphanumeric string for an annotation run in Darwin. It **must** start with a letter. Annotation runs on a particular `Dataset` or `View` can be displayed with:

**The `Dataset.annotate()` function**

When run, this function instantiates an annotation run. This means:

- 1: Creates the passed `dataset_slug` as a Darwin dataset if it doesn’t exist
- 2: Creates the passed `label_schema` (see below) as a set of annotation classes in the dataset. Existing classes will be added to the dataset, and new classes will be created
- 3: Uploads all samples added to the `Dataset` object to the Darwin dataset. This is achieved by using the `push()` [function in darwin-py](https://github.com/v7labs/darwin-py/blob/3bc1912101c51e40c138d6ee916a416d9a584065/darwin/dataset/remote_dataset_v2.py#L144))
- 4: Uploads annotations in the Voxel51 dataset that are of any class passed in the label schema (see below) - This is achieved by using the `import_annotations()` [function in darwin-py](https://github.com/v7labs/darwin-py/blob/3bc1912101c51e40c138d6ee916a416d9a584065/darwin/importer/importer.py#L668))

The `backend` argument is always `darwin`. It’s necessary because Voxel51 uses the same function for integration with other partners, in which case the value of `backend` changes

In [ ]:
dataset.list_annotation_runs()

### Annotation in Darwin take place at this stage

In [ ]:
# You can use the below code to check the status of the annotation job
results = dataset.load_annotation_results(anno_key)
results.check_status()

In [ ]:
# After annotating, send the annotations back:
dataset.load_annotations(anno_key)

The **`Dataset.load_annotations()`** function:

When run, this function concludes an annotation run. This means:

- 1: Creating an export of all items in the target dataset
- 2: Downloading the annotation ZIP file once the export is complete
- 3: Cycling through all annotations, and converting all annotations for classes **matching the original label schema class names** back to Voxel51’s own format. This means that classes added to a dataset mid-annotation run **will not be considered!** However, note that property values or attributes added to included classes mid-annotation run will be included in the export

In [ ]:
# Finally, visualise the results back in Voxel51
session = fo.launch_app(dataset)

In [ ]:
# To delete the associated V7 dataset and workflow you can use the cleanupmethod. It should be used only when you are finished with that data in V7.
results = dataset.load_annotation_results(anno_key)
results.cleanup()